# Importation of the libraries and datasets

In [1]:
# Libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(color_codes=True)

In [2]:
# FAO Dataframes

# FAO animal slaughter data
slaughter = pd.read_csv("C:/Users/shaun/OneDrive/MSc Data Analytics/CA2 Semester 1/Data/FAOSTAT_slaughtered.csv", 
                        low_memory=False)

# FAO meat production data
meat = pd.read_csv("C:/Users/shaun/OneDrive/MSc Data Analytics/CA2 Semester 1/Data/FAOSTAT_production_quantity.csv", 
                   low_memory=False)

# FAO livestock Stocking data
stock = pd.read_csv("C:/Users/shaun/OneDrive/MSc Data Analytics/CA2 Semester 1/Data/FAOSTAT_stocks.csv", 
                    low_memory=False)

# FAO live stock import / export data
imp_exp = pd.read_csv("C:/Users/shaun/OneDrive/MSc Data Analytics/CA2 Semester 1/Data/FAOSTAT_import_export.csv", 
                      low_memory=False)

In [3]:
# Functions

def calc_percent_missing(no_observations, maxobservations):
    '''This takes the number of missing and maximum observations as inputs. It
    then calculates the no of missing observations as a percentage of the total observations'''
    percent_miss = round((no_observations / maxobservations) * (100 / 1), 2)
    print(f"The percentage of missing values is:\n{percent_miss}")
    


# Exploratory data analysis

## Animal slaughter data

Here we view the head and shape of the data

In [4]:
slaughter.head()

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code (FAO),Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QCL,Crops and livestock products,255,Belgium,5320,Producing Animals/Slaughtered,867,"Meat, cattle",2000,2000,Head,832926.0,NaN,Official data
1,QCL,Crops and livestock products,255,Belgium,5320,Producing Animals/Slaughtered,867,"Meat, cattle",2001,2001,Head,873268.0,NaN,Official data
2,QCL,Crops and livestock products,255,Belgium,5320,Producing Animals/Slaughtered,867,"Meat, cattle",2002,2002,Head,932473.0,NaN,Official data
3,QCL,Crops and livestock products,255,Belgium,5320,Producing Animals/Slaughtered,867,"Meat, cattle",2003,2003,Head,853641.0,NaN,Official data
4,QCL,Crops and livestock products,255,Belgium,5320,Producing Animals/Slaughtered,867,"Meat, cattle",2004,2004,Head,842585.0,NaN,Official data


In [5]:
slaughter.shape

(3746, 14)

In [6]:
slaughter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3746 entries, 0 to 3745
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       3746 non-null   object 
 1   Domain            3746 non-null   object 
 2   Area Code (FAO)   3746 non-null   int64  
 3   Area              3746 non-null   object 
 4   Element Code      3746 non-null   int64  
 5   Element           3746 non-null   object 
 6   Item Code (FAO)   3746 non-null   int64  
 7   Item              3746 non-null   object 
 8   Year Code         3746 non-null   int64  
 9   Year              3746 non-null   int64  
 10  Unit              3746 non-null   object 
 11  Value             3654 non-null   float64
 12  Flag              1401 non-null   object 
 13  Flag Description  3746 non-null   object 
dtypes: float64(1), int64(5), object(8)
memory usage: 409.8+ KB


In [7]:
slaughter.describe()

,Area Code (FAO),Element Code,Item Code (FAO),Year Code,Year,Value
count,3746.000000,3746.000000,3746.000000,3746.000000,3746.000000,3.654000e+03
mean,153.586759,5320.452483,1036.179391,1997.905499,1997.905499,1.232273e+07
std,98.639385,0.497803,73.686268,13.784635,13.784635,5.686957e+07
min,54.000000,5320.000000,867.000000,1973.000000,1973.000000,0.000000e+00
25%,79.000000,5320.000000,1017.000000,1986.000000,1986.000000,1.230950e+04
50%,106.000000,5320.000000,1058.000000,1999.000000,1999.000000,2.321095e+05
75%,231.000000,5321.000000,1080.000000,2010.000000,2010.000000,2.915900e+06
max,351.000000,5321.000000,1163.000000,2020.000000,2020.000000,7.449179e+08


The data is in its current layout is not suitable for statistical analysis as each country is a row.
I need to melt the data so that the values for each country are in a column item type.

In [8]:
slaughter.Item.unique()

array(['Meat, cattle', 'Meat, chicken', 'Meat, duck', 'Meat, goat',
       'Meat, horse', 'Meat, pig', 'Meat, sheep', 'Meat, turkey',
       'Meat, goose and guinea fowl', 'Meat, rabbit', 'Meat, game'],
      dtype=object)

There are 11 animal categories in the slaughter data. I can remove the repetition from these item categories when I clean the data

In [9]:
slaughter.Area.unique()

array(['Belgium', 'China', 'Denmark', 'France', 'Germany', 'Ireland',
       'Italy', 'Luxembourg', 'Netherlands', 'United States of America'],
      dtype=object)

The dataset contains 8 EU countries Belgium, Denmark, France, Germany, Luxembourg, Ireland, Italy and the Netherlands. All of these countries have been member states since at least Jan 1st 1973.

Belgium, France, Germany, Luxembourg, Italy and the Netherlands founded the EU in 1957 with Ireland and Denmark Joining on the 1st January 1973.

Therefore, to ensure data comparability between these countries and the US and China, I limited the datasets time frame from 1st January 1973 to 31st December 2020 (most recent data available). This ensures that all European countries were member states of the EU at the time of analysis. 

We can confirm this as follows

In [10]:
slaughter.Year.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 1973,
       1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984,
       1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999], dtype=int64)

In [11]:
print(f"The earliest year in the dataset is {slaughter.Year.min()} and the maximum year is {slaughter.Year.max()}")

The earliest year in the dataset is 1973 and the maximum year is 2020


Next we can observe what units the slaughter data has been recorded in

In [12]:
slaughter.Unit.unique()

array(['Head', '1000 Head'], dtype=object)

Slaughter units have been reported as per "head" and per "thousand heads" of animal.

In [13]:
slaughter.Element.unique()

array(['Producing Animals/Slaughtered'], dtype=object)

All livestock in this dataset is at slaughter.

Checking for null values

In [14]:
slaughter.isnull().sum()

Domain Code            0
Domain                 0
Area Code (FAO)        0
Area                   0
Element Code           0
Element                0
Item Code (FAO)        0
Item                   0
Year Code              0
Year                   0
Unit                   0
Value                 92
Flag                2345
Flag Description       0
dtype: int64

In [15]:
calc_percent_missing(slaughter.isnull().sum(), len(slaughter))

The percentage of missing values is:
Domain Code          0.00
Domain               0.00
Area Code (FAO)      0.00
Area                 0.00
Element Code         0.00
Element              0.00
Item Code (FAO)      0.00
Item                 0.00
Year Code            0.00
Year                 0.00
Unit                 0.00
Value                2.46
Flag                62.60
Flag Description     0.00
dtype: float64


There are missing values in the "Value" and "Flag" columns.  The flag column is not important to our data analysis
so I can remove this during data preparation. However, the value column requires further investigation. We can test if these values are missing at random or not at random by filtering the values that are NaN or 0

In [16]:
# Creating a data frame of missing slaughtervalues
slaughter_miss = slaughter.loc[(slaughter['Value'] == slaughter.Value.isnull())]

In [17]:
slaughter_miss.head(50)

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code (FAO),Item,Year Code,Year,Unit,Value,Flag,Flag Description
789,QCL,Crops and livestock products,54,Denmark,5320,Producing Animals/Slaughtered,1017,"Meat, goat",2018,2018,Head,0.0,NaN,Official data
790,QCL,Crops and livestock products,54,Denmark,5320,Producing Animals/Slaughtered,1017,"Meat, goat",2019,2019,Head,0.0,NaN,Official data
791,QCL,Crops and livestock products,54,Denmark,5320,Producing Animals/Slaughtered,1017,"Meat, goat",2020,2020,Head,0.0,NaN,Official data
2148,QCL,Crops and livestock products,104,Ireland,5320,Producing Animals/Slaughtered,1017,"Meat, goat",2018,2018,Head,0.0,NaN,Official data
2149,QCL,Crops and livestock products,104,Ireland,5320,Producing Animals/Slaughtered,1017,"Meat, goat",2019,2019,Head,0.0,NaN,Official data
2150,QCL,Crops and livestock products,104,Ireland,5320,Producing Animals/Slaughtered,1017,"Meat, goat",2020,2020,Head,0.0,NaN,Official data
2844,QCL,Crops and livestock products,256,Luxembourg,5321,Producing Animals/Slaughtered,1058,"Meat, chicken",2018,2018,1000 Head,0.0,NaN,Official data
2845,QCL,Crops and livestock products,256,Luxembourg,5321,Producing Animals/Slaughtered,1058,"Meat, chicken",2019,2019,1000 Head,0.0,NaN,Official data
2846,QCL,Crops and livestock products,256,Luxembourg,5321,Producing Animals/Slaughtered,1058,"Meat, chicken",2020,2020,1000 Head,0.0,NaN,Official data
2847,QCL,Crops and livestock products,256,Luxembourg,5321,Producing Animals/Slaughtered,1069,"Meat, duck",2018,2018,1000 Head,0.0,NaN,Official data


In [18]:
# Investigating if the missing data is at random or not at random by checking if it's specific animal categories or not
slaughter_miss.Item.unique()

array(['Meat, goat', 'Meat, chicken', 'Meat, duck', 'Meat, horse',
       'Meat, turkey'], dtype=object)

In [19]:
slaughter_miss.Value.unique()

array([0.])

In [20]:
# Investigating if the missing data is at random or not at random by checking if it's specific country's or not
slaughter_miss.Area.unique()

array(['Denmark', 'Ireland', 'Luxembourg', 'Netherlands'], dtype=object)

In [21]:
slaughter_miss.Year.unique()

array([2018, 2019, 2020, 2000, 2001, 1973, 1974, 1975, 1976], dtype=int64)

This shows that the missing data is NOT at random, certain countries did not submit data for certain animal categories on certain years. I will deal with this during data preparation.

## Meat production data

In [22]:
meat.head()

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code (FAO),Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QCL,Crops and livestock products,255,Belgium,5510,Production,867,"Meat, cattle",2000,2000,tonnes,275360.0,NaN,Official data
1,QCL,Crops and livestock products,255,Belgium,5510,Production,867,"Meat, cattle",2001,2001,tonnes,285250.0,NaN,Official data
2,QCL,Crops and livestock products,255,Belgium,5510,Production,867,"Meat, cattle",2002,2002,tonnes,305388.0,NaN,Official data
3,QCL,Crops and livestock products,255,Belgium,5510,Production,867,"Meat, cattle",2003,2003,tonnes,275170.0,NaN,Official data
4,QCL,Crops and livestock products,255,Belgium,5510,Production,867,"Meat, cattle",2004,2004,tonnes,280931.0,NaN,Official data


In [23]:
meat.shape

(3940, 14)

In [24]:
meat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3940 entries, 0 to 3939
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       3940 non-null   object 
 1   Domain            3940 non-null   object 
 2   Area Code (FAO)   3940 non-null   int64  
 3   Area              3940 non-null   object 
 4   Element Code      3940 non-null   int64  
 5   Element           3940 non-null   object 
 6   Item Code (FAO)   3940 non-null   int64  
 7   Item              3940 non-null   object 
 8   Year Code         3940 non-null   int64  
 9   Year              3940 non-null   int64  
 10  Unit              3940 non-null   object 
 11  Value             3862 non-null   float64
 12  Flag              1298 non-null   object 
 13  Flag Description  3940 non-null   object 
dtypes: float64(1), int64(5), object(8)
memory usage: 431.1+ KB


In [25]:
meat.describe()

,Area Code (FAO),Element Code,Item Code (FAO),Year Code,Year,Value
count,3940.000000,3940.0,3940.000000,3940.000000,3940.000000,3.862000e+03
mean,154.790102,5510.0,1042.574873,1997.820051,1997.820051,1.300366e+06
std,99.951755,0.0,77.053159,13.754285,13.754285,4.658044e+06
min,54.000000,5510.0,867.000000,1973.000000,1973.000000,0.000000e+00
25%,79.000000,5510.0,1017.000000,1986.000000,1986.000000,4.966500e+03
50%,106.000000,5510.0,1058.000000,1999.000000,1999.000000,6.400000e+04
75%,231.000000,5510.0,1080.000000,2010.000000,2010.000000,6.888240e+05
max,351.000000,5510.0,1163.000000,2020.000000,2020.000000,5.766187e+07


The data is in its current layout is not suitable for statistical analysis as each country is a row.
I need to melt the data so that the values for each country are in a column item type.

In [26]:
meat.Item.unique()

array(['Meat, cattle', 'Meat, chicken', 'Meat, duck', 'Meat, goat',
       'Meat, horse', 'Meat, pig', 'Meat, sheep', 'Meat, turkey',
       'Meat, game', 'Meat, goose and guinea fowl', 'Meat, rabbit'],
      dtype=object)

There are 11 animal categories in the production data. I can remove the repetition from these item categories when I clean the data

In [27]:
meat.Area.unique()

array(['Belgium', 'China', 'Denmark', 'France', 'Germany', 'Ireland',
       'Italy', 'Luxembourg', 'Netherlands', 'United States of America'],
      dtype=object)

As above:

The dataset contains 8 EU countries Belgium, Denmark, France, Germany, Luxembourg, Ireland, Italy and the Netherlands. All of these countries have been member states since at least Jan 1st 1973.

Belgium, France, Germany, Luxembourg, Italy and the Netherlands founded the EU in 1957 with Ireland and Denmark Joining on the 1st January 1973.

Therefore, to ensure data comparability between these countries and the US and China, I limited the datasets time frame from 1st January 1973 to 31st December 2020 (most recent data available). This ensures that all European countries were member states of the EU at the time of analysis. 

We can confirm this as follows

In [28]:
meat.Year.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 1973,
       1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984,
       1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999], dtype=int64)

In [29]:
print(f"The earliest year in the dataset is {meat.Year.min()} and the maximum year is {meat.Year.max()}")

The earliest year in the dataset is 1973 and the maximum year is 2020


Next we can observe what units the meat production data has been recorded in

In [30]:
meat.Unit.unique()

array(['tonnes'], dtype=object)

Production data units have been reported in "tonnes".

In [31]:
meat.Element.unique()

array(['Production'], dtype=object)

All livestock in this dataset is at production.

Checking for null values

In [32]:
meat.isnull().sum()

Domain Code            0
Domain                 0
Area Code (FAO)        0
Area                   0
Element Code           0
Element                0
Item Code (FAO)        0
Item                   0
Year Code              0
Year                   0
Unit                   0
Value                 78
Flag                2642
Flag Description       0
dtype: int64

In [33]:
calc_percent_missing(meat.isnull().sum(), len(meat))

The percentage of missing values is:
Domain Code          0.00
Domain               0.00
Area Code (FAO)      0.00
Area                 0.00
Element Code         0.00
Element              0.00
Item Code (FAO)      0.00
Item                 0.00
Year Code            0.00
Year                 0.00
Unit                 0.00
Value                1.98
Flag                67.06
Flag Description     0.00
dtype: float64


There are missing values in the "Value" and "Flag" columns.  The flag column is not important to our data analysis
so I can remove this during data preparation. However, the value column requires further investigation. We can test if these values are missing at random or not at random by filtering the values that are NaN or 0

In [34]:
# Creating a data frame of missing meat production
meat_miss = meat.loc[(meat['Value'] == meat.Value.isnull())]

In [35]:
meat_miss.head(50)

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code (FAO),Item,Year Code,Year,Unit,Value,Flag,Flag Description
883,QCL,Crops and livestock products,54,Denmark,5510,Production,1017,"Meat, goat",2018,2018,tonnes,0.0,NaN,Official data
884,QCL,Crops and livestock products,54,Denmark,5510,Production,1017,"Meat, goat",2019,2019,tonnes,0.0,NaN,Official data
885,QCL,Crops and livestock products,54,Denmark,5510,Production,1017,"Meat, goat",2020,2020,tonnes,0.0,NaN,Official data
1151,QCL,Crops and livestock products,54,Denmark,5510,Production,1080,"Meat, turkey",2018,2018,tonnes,0.0,NaN,Official data
1152,QCL,Crops and livestock products,54,Denmark,5510,Production,1080,"Meat, turkey",2019,2019,tonnes,0.0,NaN,Official data
1153,QCL,Crops and livestock products,54,Denmark,5510,Production,1080,"Meat, turkey",2020,2020,tonnes,0.0,NaN,Official data
1862,QCL,Crops and livestock products,79,Germany,5510,Production,1017,"Meat, goat",2018,2018,tonnes,0.0,NaN,Official data
1863,QCL,Crops and livestock products,79,Germany,5510,Production,1017,"Meat, goat",2019,2019,tonnes,0.0,NaN,Official data
1864,QCL,Crops and livestock products,79,Germany,5510,Production,1017,"Meat, goat",2020,2020,tonnes,0.0,NaN,Official data
2293,QCL,Crops and livestock products,104,Ireland,5510,Production,1017,"Meat, goat",2018,2018,tonnes,0.0,NaN,Official data


In [36]:
# Investigating if the missing data is at random or not at random by checking if it's specific animal categories or not
meat_miss.Item.unique()

array(['Meat, goat', 'Meat, turkey', 'Meat, chicken', 'Meat, duck',
       'Meat, horse', 'Meat, game'], dtype=object)

In [37]:
meat_miss.Value.unique()

array([0.])

In [38]:
# Investigating if the missing data is at random or not at random by checking if it's specific country's or not
meat_miss.Area.unique()

array(['Denmark', 'Germany', 'Ireland', 'Luxembourg', 'Netherlands'],
      dtype=object)

In [39]:
meat_miss.Year.unique()

array([2018, 2019, 2020, 2000, 2001, 1986, 2005], dtype=int64)

This shows that the missing data is NOT at random, certain countries did not submit data for certain animal categories on certain years. I will deal with this during data preparation.

In addition, it shows that the missing categories and years are similar to the Slaughter data. There may have been no requirement to report for these categories on these years.

## Livestock stocking data

In [40]:
stock.head()

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code (FAO),Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QCL,Crops and livestock products,255,Belgium,5111,Stocks,866,Cattle,2000,2000,Head,3041560.0,NaN,Official data
1,QCL,Crops and livestock products,255,Belgium,5111,Stocks,866,Cattle,2001,2001,Head,3037760.0,NaN,Official data
2,QCL,Crops and livestock products,255,Belgium,5111,Stocks,866,Cattle,2002,2002,Head,2891260.0,NaN,Official data
3,QCL,Crops and livestock products,255,Belgium,5111,Stocks,866,Cattle,2003,2003,Head,2778077.0,NaN,Official data
4,QCL,Crops and livestock products,255,Belgium,5111,Stocks,866,Cattle,2004,2004,Head,2738648.0,NaN,Official data


In [41]:
stock.shape

(3735, 14)

In [42]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3735 entries, 0 to 3734
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       3735 non-null   object 
 1   Domain            3735 non-null   object 
 2   Area Code (FAO)   3735 non-null   int64  
 3   Area              3735 non-null   object 
 4   Element Code      3735 non-null   int64  
 5   Element           3735 non-null   object 
 6   Item Code (FAO)   3735 non-null   int64  
 7   Item              3735 non-null   object 
 8   Year Code         3735 non-null   int64  
 9   Year              3735 non-null   int64  
 10  Unit              3735 non-null   object 
 11  Value             3670 non-null   float64
 12  Flag              1103 non-null   object 
 13  Flag Description  3735 non-null   object 
dtypes: float64(1), int64(5), object(8)
memory usage: 408.6+ KB


In [43]:
stock.describe()

,Area Code (FAO),Element Code,Item Code (FAO),Year Code,Year,Value
count,3735.000000,3735.000000,3735.000000,3735.00000,3735.00000,3.670000e+03
mean,154.917269,5111.448728,1033.941098,1997.31593,1997.31593,1.389960e+07
std,98.436050,0.497431,73.118241,13.67837,13.67837,5.001213e+07
min,54.000000,5111.000000,866.000000,1973.00000,1973.00000,2.000000e+00
25%,79.000000,5111.000000,1016.000000,1985.00000,1985.00000,8.278250e+03
50%,106.000000,5111.000000,1057.000000,1998.00000,1998.00000,1.960655e+05
75%,231.000000,5112.000000,1079.000000,2009.00000,2009.00000,6.134275e+06
max,351.000000,5112.000000,1140.000000,2020.00000,2020.00000,4.867429e+08


The data is in its current layout is not suitable for statistical analysis as each country is a row.
I need to melt the data so that the values for each country are in a column item type.

In [44]:
stock.Item.unique()

array(['Cattle', 'Chickens', 'Ducks', 'Geese and guinea fowls', 'Goats',
       'Horses', 'Pigs', 'Rabbits and hares', 'Sheep', 'Turkeys'],
      dtype=object)

There are 10 animal categories in the stocking data.

In [45]:
stock.Area.unique()

array(['Belgium', 'China', 'Denmark', 'France', 'Germany', 'Ireland',
       'Italy', 'Luxembourg', 'Netherlands', 'United States of America'],
      dtype=object)

As above:

The dataset contains 8 EU countries Belgium, Denmark, France, Germany, Luxembourg, Ireland, Italy and the Netherlands. All of these countries have been member states since at least Jan 1st 1973.

Belgium, France, Germany, Luxembourg, Italy and the Netherlands founded the EU in 1957 with Ireland and Denmark Joining on the 1st January 1973.

Therefore, to ensure data comparability between these countries and the US and China, I limited the datasets time frame from 1st January 1973 to 31st December 2020 (most recent data available). This ensures that all European countries were member states of the EU at the time of analysis. 

We can confirm this as follows

In [46]:
stock.Year.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 1973,
       1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984,
       1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999], dtype=int64)

In [47]:
print(f"The earliest year in the dataset is {stock.Year.min()} and the maximum year is {stock.Year.max()}")

The earliest year in the dataset is 1973 and the maximum year is 2020


Next we can observe what units the livestock stocking data has been recorded in

In [48]:
stock.Unit.unique()

array(['Head', '1000 Head'], dtype=object)

Socking data units have been reported in "heads" and "1000 heads".

In [49]:
stock.Element.unique()

array(['Stocks'], dtype=object)

All livestock in this dataset is at production.

Checking for null values

In [50]:
stock.isnull().sum()

Domain Code            0
Domain                 0
Area Code (FAO)        0
Area                   0
Element Code           0
Element                0
Item Code (FAO)        0
Item                   0
Year Code              0
Year                   0
Unit                   0
Value                 65
Flag                2632
Flag Description       0
dtype: int64

In [51]:
calc_percent_missing(stock.isnull().sum(), len(stock))

The percentage of missing values is:
Domain Code          0.00
Domain               0.00
Area Code (FAO)      0.00
Area                 0.00
Element Code         0.00
Element              0.00
Item Code (FAO)      0.00
Item                 0.00
Year Code            0.00
Year                 0.00
Unit                 0.00
Value                1.74
Flag                70.47
Flag Description     0.00
dtype: float64


There are missing values in the "Value" and "Flag" columns.  The flag column is not important to our data analysis
so I can remove this during data preparation. However, the value column requires further investigation. We can test if these values are missing at random or not at random by filtering the values that are NaN or 0

In [52]:
stock.columns

Index(['Domain Code', 'Domain', 'Area Code (FAO)', 'Area', 'Element Code',
       'Element', 'Item Code (FAO)', 'Item', 'Year Code', 'Year', 'Unit',
       'Value', 'Flag', 'Flag Description'],
      dtype='object')

In [53]:
# Creating a data frame of missing stocking densities
stock_miss = stock.loc[(stock['Value'] == stock.Value.isnull())]

In [54]:
stock_miss.head(50)

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code (FAO),Item,Year Code,Year,Unit,Value,Flag,Flag Description


In [55]:
# Investigating if the missing data is at random or not at random by checking if it's specific animal categories or not
stock_miss.Item.unique()

array([], dtype=object)

In [56]:
stock_miss.Value.unique()

array([], dtype=float64)

In [57]:
# Investigating if the missing data is at random or not at random by checking if it's specific country's or not
stock_miss.Area.unique()

array([], dtype=object)

In [58]:
stock_miss.Year.unique()

array([], dtype=int64)

## Import export data

In [59]:
imp_exp.head()

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code (FAO),Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,TCL,Crops and livestock products,255,Belgium,5608,Import Quantity,866,Cattle,2000,2000,Head,59395.0,NaN,Official data
1,TCL,Crops and livestock products,255,Belgium,5608,Import Quantity,866,Cattle,2001,2001,Head,44232.0,NaN,Official data
2,TCL,Crops and livestock products,255,Belgium,5608,Import Quantity,866,Cattle,2002,2002,Head,61054.0,NaN,Official data
3,TCL,Crops and livestock products,255,Belgium,5608,Import Quantity,866,Cattle,2003,2003,Head,85727.0,NaN,Official data
4,TCL,Crops and livestock products,255,Belgium,5608,Import Quantity,866,Cattle,2004,2004,Head,100891.0,NaN,Official data


In [60]:
imp_exp.shape

(14411, 14)

In [61]:
imp_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14411 entries, 0 to 14410
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       14411 non-null  object 
 1   Domain            14411 non-null  object 
 2   Area Code (FAO)   14411 non-null  int64  
 3   Area              14411 non-null  object 
 4   Element Code      14411 non-null  int64  
 5   Element           14411 non-null  object 
 6   Item Code (FAO)   14411 non-null  int64  
 7   Item              14411 non-null  object 
 8   Year Code         14411 non-null  int64  
 9   Year              14411 non-null  int64  
 10  Unit              14411 non-null  object 
 11  Value             14263 non-null  float64
 12  Flag              2446 non-null   object 
 13  Flag Description  14411 non-null  object 
dtypes: float64(1), int64(5), object(8)
memory usage: 1.5+ MB


In [62]:
imp_exp.describe()

,Area Code (FAO),Element Code,Item Code (FAO),Year Code,Year,Value
count,14411.000000,14411.000000,14411.000000,14411.000000,14411.000000,1.426300e+04
mean,154.272778,5764.227257,1023.589480,1998.889182,1998.889182,1.764494e+05
std,95.451054,150.154608,70.081096,14.211274,14.211274,7.850559e+05
min,54.000000,5608.000000,866.000000,1973.000000,1973.000000,0.000000e+00
25%,79.000000,5610.000000,976.000000,1987.000000,1987.000000,2.640000e+02
50%,106.000000,5622.000000,1034.000000,2000.000000,2000.000000,5.808000e+03
75%,231.000000,5910.000000,1079.000000,2012.000000,2012.000000,6.571350e+04
max,351.000000,5922.000000,1096.000000,2020.000000,2020.000000,1.589366e+07


The data is in its current layout is not suitable for statistical analysis as each country is a row.
I need to melt the data so that the values for each country are in a column item type.

In [63]:
imp_exp.Item.unique()

array(['Cattle', 'Chickens', 'Ducks', 'Goats', 'Horses', 'Pigs', 'Sheep',
       'Turkeys'], dtype=object)

There are 8 animal categories in the import export data. I can remove the repetition from these item categories when I clean the data

In [64]:
imp_exp.Area.unique()

array(['Belgium', 'China', 'Denmark', 'France', 'Germany', 'Ireland',
       'Italy', 'Luxembourg', 'Netherlands', 'United States of America'],
      dtype=object)

As above:

The dataset contains 8 EU countries Belgium, Denmark, France, Germany, Luxembourg, Ireland, Italy and the Netherlands. All of these countries have been member states since at least Jan 1st 1973.

Belgium, France, Germany, Luxembourg, Italy and the Netherlands founded the EU in 1957 with Ireland and Denmark Joining on the 1st January 1973.

Therefore, to ensure data comparability between these countries and the US and China, I limited the datasets time frame from 1st January 1973 to 31st December 2020 (most recent data available). This ensures that all European countries were member states of the EU at the time of analysis. 

We can confirm this as follows

In [65]:
imp_exp.Year.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 1973,
       1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984,
       1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999], dtype=int64)

In [66]:
print(f"The earliest year in the dataset is {imp_exp.Year.min()} and the maximum year is {imp_exp.Year.max()}")

The earliest year in the dataset is 1973 and the maximum year is 2020


Next we can observe what units the import export data has been recorded in

In [67]:
imp_exp.Unit.unique()

array(['Head', 'tonnes', '1000 US$', '1000 Head'], dtype=object)

Import export data has been recorded in in "tonnes", "heads", "1000 US $" and "1000 heads".

In [68]:
imp_exp.Element.unique()

array(['Import Quantity', 'Import Value', 'Export Quantity',
       'Export Value'], dtype=object)

All livestock in this dataset is at production.

Checking for null values

In [69]:
imp_exp.isnull().sum()

Domain Code             0
Domain                  0
Area Code (FAO)         0
Area                    0
Element Code            0
Element                 0
Item Code (FAO)         0
Item                    0
Year Code               0
Year                    0
Unit                    0
Value                 148
Flag                11965
Flag Description        0
dtype: int64

In [70]:
calc_percent_missing(imp_exp.isnull().sum(), len(imp_exp))

The percentage of missing values is:
Domain Code          0.00
Domain               0.00
Area Code (FAO)      0.00
Area                 0.00
Element Code         0.00
Element              0.00
Item Code (FAO)      0.00
Item                 0.00
Year Code            0.00
Year                 0.00
Unit                 0.00
Value                1.03
Flag                83.03
Flag Description     0.00
dtype: float64


There are missing values in the "Value" and "Flag" columns.  The flag column is not important to our data analysis
so I can remove this during data preparation. However, the value column requires further investigation. We can test if these values are missing at random or not at random by filtering the values that are NaN or 0

In [71]:
# Creating a data frame of missing import export data 
imp_exp_miss = imp_exp.loc[(imp_exp['Value'] == imp_exp.Value.isnull())]

In [72]:
meat_miss.head(50)

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code (FAO),Item,Year Code,Year,Unit,Value,Flag,Flag Description
883,QCL,Crops and livestock products,54,Denmark,5510,Production,1017,"Meat, goat",2018,2018,tonnes,0.0,NaN,Official data
884,QCL,Crops and livestock products,54,Denmark,5510,Production,1017,"Meat, goat",2019,2019,tonnes,0.0,NaN,Official data
885,QCL,Crops and livestock products,54,Denmark,5510,Production,1017,"Meat, goat",2020,2020,tonnes,0.0,NaN,Official data
1151,QCL,Crops and livestock products,54,Denmark,5510,Production,1080,"Meat, turkey",2018,2018,tonnes,0.0,NaN,Official data
1152,QCL,Crops and livestock products,54,Denmark,5510,Production,1080,"Meat, turkey",2019,2019,tonnes,0.0,NaN,Official data
1153,QCL,Crops and livestock products,54,Denmark,5510,Production,1080,"Meat, turkey",2020,2020,tonnes,0.0,NaN,Official data
1862,QCL,Crops and livestock products,79,Germany,5510,Production,1017,"Meat, goat",2018,2018,tonnes,0.0,NaN,Official data
1863,QCL,Crops and livestock products,79,Germany,5510,Production,1017,"Meat, goat",2019,2019,tonnes,0.0,NaN,Official data
1864,QCL,Crops and livestock products,79,Germany,5510,Production,1017,"Meat, goat",2020,2020,tonnes,0.0,NaN,Official data
2293,QCL,Crops and livestock products,104,Ireland,5510,Production,1017,"Meat, goat",2018,2018,tonnes,0.0,NaN,Official data


In [73]:
# Investigating if the missing data is at random or not at random by checking if it's specific animal categories or not
imp_exp_miss.Item.unique()

array(['Ducks', 'Goats', 'Turkeys', 'Horses', 'Sheep', 'Cattle', 'Pigs',
       'Chickens'], dtype=object)

In [74]:
imp_exp_miss.Value.unique()

array([0.])

In [75]:
# Investigating if the missing data is at random or not at random by checking if it's specific country's or not
imp_exp_miss.Area.unique()

array(['Belgium', 'China', 'Denmark', 'France', 'Germany', 'Ireland',
       'Italy', 'Luxembourg', 'Netherlands', 'United States of America'],
      dtype=object)

In [76]:
imp_exp_miss.Year.unique()

array([2019, 2020, 2014, 2016, 2015, 2009, 2010, 2011, 2012, 2013, 1973,
       1974, 1975, 1976, 2018, 1977, 1978, 1981, 2005, 2006, 2008, 2017,
       1979, 1980, 1982, 1984, 1983, 1985, 2002, 1986, 1993, 1994, 1995,
       1997, 1998, 1987, 1988, 1989, 1990, 1996, 1999, 2000, 2001, 2003,
       2004, 2007, 1991, 1992], dtype=int64)

## Summary of EDA

Missing values account for a small number of the data in the Value columns of all datasets. Missing values have been recorded in two manners, some as "Zero" and others as a blank (NaN). The missing data is also not at random, it appears specific countries did not submit specific categories on specific years. The only other column with missing data is the flag data which is not useful to the analysis. 

The data will also need to be melted into each country having its values in a column in order to be able to accurately perform statistical analysis.

# NAN's do not show up on is.null filter, it is filtering the Zero values

# Data preparation 

## Removing the Null values

In [ ]:
# removing NANs and filtering out "Zero"

slaughter_clean =

meat_clean =

stock_clean =

imp_exp_clean =

Importing the country codes file and using it to map country name to each code in the dataframes using python melt

In [77]:
country_codes = pd.read_csv("country_codes.tsv", sep='\t')
country_codes.head()

# This can be used if you get data only with country names, it allows you to
# map the country name to the code on new data sets and then you can make
# your visualisation

FileNotFoundError: [Errno 2] No such file or directory: 'country_codes.tsv'

In [ ]:
df = pd.melt(slaughter, id_vars=['geo'], var_name="Year", 
             value_name="Number")

In [ ]:
df.info()

In [ ]:
df.head()